
<div style="background-color: #ccffcc; padding: 10px;">
    <h1> Tutorial 5 Part 2 </h1> 
    <h2> Introduction to Generative Adversarial Networks </h2>
</div>      

# Overview

This Jupyter notebook will walk you through building a simple GAN model and training it on the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) using [PyTorch](https://pytorch.org/). PyTorch is a very powerful library written in Python and C++ that facilitates the building and training of neural networks.

This tutorial is in 2 parts this is **Part 2**.

**Part 1** was adapted from Caitlin Howarth's Presentation and tutorial as part of the LIFD Work shop series. Some of the code and explanations from this tutorial came from [Aleksa Gordić's pyctorch-gans repository](https://github.com/gordicaleksa/pytorch-gans). It contains some more in-depth explanations, as well as examples of more advanced GAN models than we covered here.

**Part 2** is an applied Earth Science example based on a [Met Office](https://www.metoffice.gov.uk/) example tropical cyclone example using [Unified Model](https://www.metoffice.gov.uk/research/approach/modelling-systems/unified-model/index) output [1]. The code and scripts used are taken from the [MetOffice/ML-TC](https://github.com/MetOffice/ML-TC) repository. GANs are allready being applied to Earth Science applications such as [improving precipitation nowcasting](https://www.nature.com/articles/s41586-021-03854-z), this example will use similar prinicals to explore tropical cyclones.  


**PART 2 uses a much larger dataset (10GB) that can not be run on BINDER or GOOGLE CO-LAB, Gerating the dataset will take up to 2 hours on a resonably powerful machine and training the dataset will take up to X hours with GPUS**

**Petrained models have been saved for both tutorial parts so if you do not have GPUs available then you can still run the turorial in a reasonable timeframe**


## The very basics

If you know nothing about neural networks there is a [toy neural network python code example](https://github.com/cemac/LIFD_ENV_ML_NOTEBOOKS/tree/main/ToyNeuralNetwork) included in the [LIFD ENV ML Notebooks Repository]( https://github.com/cemac/LIFD_ENV_ML_NOTEBOOKS). Creating a 2 layer neural network to illustrate the fundamentals of how Neural Networks work and the equivlent code using the python machine learning library [keras](https://keras.io/). 

## Recommended reading 


* [Introduction to Neural Networks]()
* [Introduction to Generative Adversarial Newtworks](https://towardsdatascience.com/fundamentals-of-generative-adversarial-networks-b7ca8c34f0bc)
* [Auto Encoders](https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798)
* [Neural Networks for Regression Problems](https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33)
* [Digitizing Sketches of the Earth’s Surface with the AI Behind Deep Fakes (IBM Earth Science Example](https://www.ibm.com/blogs/research/2019/06/good-gans/)
* [DCGAN Tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)


## References

* [1] *Steptoe, H., Savage, N.H., Sadri, S. et al. Tropical cyclone simulations over Bangladesh at convection permitting 4.4 km & 1.5 km resolution. Sci Data 8, 62 (2021). [https://doi.org/10.1038/s41597-021-00847-5](https://doi.org/10.1038/s41597-021-00847-5)*
* [2] *Ravuri, S., Lenc, K., Willson, M. et al. Skilful precipitation nowcasting using deep generative models of radar. Nature 597, 672–677 (2021). [https://doi.org/10.1038/s41586-021-03854-z](https://doi.org/10.1038/s41586-021-03854-z)* 

<div style="background-color: #cce5ff; padding: 10px;">

The below cell calls a python script that will download all the necessary [UM]() data and make set of netcdf files containing variables for 11 cyclones: 
    
**BOB01**, **BOB07**, **TC01B**, **AKASH**, **SIDR**, **RASHMI**, **AILA**, **VIYARU**, **ROANU**, **MORA** and **FANI**   

    
* **fg** - wind_speed_of_gust
* **hur** - relative_humidity
* **psl** - air_pressure_at_sea_level
* **rsds** - surface_downwelling_shortwave_flux_in_air
* **rsnds** - net_down_surface_sw_flux_corrected
* **tas** - air_temperature
* **ua** - x_wind
* **va** - y_wind
* **wbpt** - wet_bulb_potential_temperature
* **zg** - geopotential_height    

</div>

In [ ]:
%%bash 

cd ML-TC/01-basic-GAN/
./getbengaldata.sh 

<hr>


<div style="background-color: #e6ccff; padding: 10px;">
    
<h1> Machine Learning Theory </h1>
 
Generative Adversarial Newtworks are a pair of Neural Networks that work to generate rather than predict. The example you've probably see in [mainstream media](https://www.artificialintelligence-news.com/2022/05/09/kendrick-lamar-uses-deepfakes-in-latest-music-video/) is deepfakes, where the network has been trained to produce images of humans that are so good even humans can't tell the difference  
    
Generative Adversarial Newtworks are made up of two [Neural Networks](): a generator and a discriminator. Both networks are equally bad at their respective tasks initially. As training continues, both networks compete with one another to get better at their respective tasks. Eventually, the generator becomes so good that humans can’t tell the difference (hopefully!)
    
**The Generator**

<a href="">
<img src="figs/generator.png">
</a>

The generator takes a latent code as an input, and then decodes that vector into an output.

Different latent codes should result in different outputs.

In the original implementation of GANs, the generator never actually sees the training dataset, and is solely guided by the discriminator. 

    
**The Discriminator**    
<a href="">
<img src="figs/discriminator.png">
</a>

    
The discriminator is just a simple regression network and outputs a score from 0 (fake) to 1 (real).

Values closer to 0 or 1 represent increased certainty that a sample is fake or real respectively.
    
**Loss Functions**

Adversarial loss is essentially a minimax game, as proposed by Ian Goodfellow et al. [1]

The discriminator attempts to maximise the likelihood of correctly predicting real samples as real, and fake samples as fake.

The generator attempts to maximise the likelihood of the discriminator predicting that the fake samples it generates are real.

* The discriminator loss is defined as: , where  is the discriminator,  is the generator,  is a sample from the real distribution and  is a latent vector given to the generator.

* The generator loss is defined as: , where  is the discriminator,  is the generator, and  is a latent vector given to the generator.
    
References:
    
* [1] Goodfellow, I.J., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., Courville, A. and Bengio, Y. 2014. Generative Adversarial Networks. [arXiv:1406.2661](https://arxiv.org/abs/1406.2661)  [cs, stat].

</div>    
  

<div style="background-color: #cce5ff; padding: 10px;">

<h1> Python </h1>

Basic python knowledge is assumed for this tutorial. For this tutorial the main machine learning library we'll be working [PyTorch](https://pytorch.org/). Python specific information will be colour coded blue.
 
    
## PyTorch
    
[PyTorch](https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e) is an open source deep learning framework Pytorch is very *pythonic*, so if you are used to coding in python using PyTorch should come naturally. 

    
References:
    
* [https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e](https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e)
*    
    
</div>
    
<hr>



<div style="background-color: #ffffcc; padding: 10px;">
    
<h1> Requirements </h1>

These notebooks should run 

<h2> Python Packages: </h2>

* Python 3
* PyTorch
* time
* os
* numpy
* matplotlib=3.0 


<h2> Data Requirements</h2>

This notebook referes to some data included in the git hub repositroy
    
</div>

**Contents:**

1. [Download MNIST Dataset](#Download-MNIST-Dataset)
2. [Building the GAN](#Building-the-GAN)
3. [Preparing to train](#Preparing-to-train)
4. [Training](#Training)
5. [Evaluating the GAN](#Evaluating-the-GAN)
6. [Example Application to Earth Sciences](Example-Application-to-Earth-Sciences) 

<div style="background-color: #cce5ff; padding: 10px;">

**Import python modules**

</div>

In [48]:
%matplotlib inline

In [23]:
import os, pathlib
import sys
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import random


<div style="background-color: #cce5ff; padding: 10px;">

Check if GPU's available. This code will run on standard CPU's too just a lots slower.
    
If no GPU's are detected, then pretrained model weights will be used set by the flag
    
```python
train_local=False
```

**If you get a message saying no GPU available but you think there should be this means you have not managed to compile a GPU version of PyTorch and need to follow the instructions in [PyTorchCondaRecipe.md](PyTorchCondaRecipe.md)**    
    
</div>

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  train_local=True
  GPUs=True  
  print(torch.cuda.get_device_name(device))
  print('GPUs found Setting train_local to True')
else:
  train_local=False
  GPUs=False
  print("WARNING: No GPU available. Setting train_local to False")

Quadro P1000
GPUs found Setting train_local to True


<div style="background-color: #cce5ff; padding: 10px;">

If you want to override this behaviour and train your own network if you don't mind waiting or if you have GPUS available but don't want to wait 20 mins training your own network later you can the the next cell to:

```python
override_train_local=True
```

</div>

In [25]:
# Set as override_train_local=False to maintain default behaviour for your hardware
override_train_local=False

In [26]:
if override_train_local :
    if GPUs:
        train_local=False  
        print('WARNING SETTING TRAIN LOCAL TO FALSE, PRETRAINED MODEL WILL BE USED')
    else:
        train_local=True
        print('WARNING SETTING TRAIN LOCAL TO TRUE, WILL TRAIN MODEL OVER A LONG TIME ON CPUS')
else: 
    print('Default behaviour')

Default behaviour


In [27]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Number of workers for dataloader
workers = 0

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

In [53]:
base_path='.'
data_path='ML-TC/01-basic-GAN/Data/'
num=0
for dir in [name for name in os.listdir(base_path)]:
    # print(dir)
    if dir.isnumeric():
        num=max(num,int(dir))
save_path=str(num+1)+"/"
os.mkdir(save_path)
sys.stdout = open(save_path+'output.txt', 'w+')
print(save_path)
t = open(save_path+"text.txt", "a")

In [58]:
save_path

'6/'

In [57]:
 t.write("Run starting")
# Set random seed for reproducibility
manualSeed = 357
# manualSeed = random.randint(1, 10000) # use if you want new results
t.write("Random Seed: "+str(manualSeed))
t.close()
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# path=str(pathlib.Path(__file__).parent)+"/Data/"


In [62]:
data

[]

In [60]:
data=[]
for f in os.listdir(data_path):
    print(f)
    if f.endswith(".npz") and 'A' in f:
        t = open(save_path+"text.txt", "a")
        t.write(f)
        t.close()
        datapoint=np.load(data_path+f, allow_pickle=True)
        for x in datapoint['arr_0']:
            # print(len(data))
            # print(x.shape)
            s=x.shape
            if s.count(s[0]) == len(s) and np.count_nonzero(~np.isnan(x)) > 3000:
                # print(x.shape)
                data.append(torch.from_numpy(np.nan_to_num(x)))
# print(len(data))
random.shuffle(data)

In [64]:
for x in data:
    plt.imshow(x, cmap='gray', vmin=0, vmax=255)
    plt.show()

<div style="background-color: #cce5ff; padding: 10px;">
PyTorch uses a DataLoader wrapper around a dataset in order to iterate through the dataset. It's where we specify the number of samples per batch, as well as whether to shuffle the data. drop_last ensures that any batch that isn't at least 128 samples gets dropped.
</div>

In [ ]:
# # Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                        shuffle=True, num_workers=workers)

# # Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
data=torch.stack(data)
# print(data.max())
# print(data.min())
data=data/(data.max()/2)-1
# print(data.max())
# print(data.min())
# transform=transforms.Compose([transforms.Resize(image_size),
#                             transforms.CenterCrop(image_size),
#                             transforms.ToTensor(),
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                         ]
dataset=TensorDataset(data)
# # We can use an image folder dataset the way we have it setup.
# # Create the dataset
# dataset = dset.ImageFolder(root=dataroot,
#                         transform=transforms.Compose([
#                             transforms.Resize(image_size),
#                             transforms.CenterCrop(image_size),
#                             transforms.ToTensor(),
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                         ]))

In [ ]:
# Plot some training images
#real_batch = next(iter(dataloader))

In [ ]:
for batch in dataloader:
     print("g")
     plt.figure(figsize=(8,8))
     plt.axis("off")
     plt.title("Training Images")
     plt.imshow(np.transpose(vutils.make_grid(batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# Building the GAN

<div style="background-color: #ccffcc; padding: 10px;">
Recall that the generator has a latent space from which we draw samples from by input random vectors. Let's define it as a vector with 100 elements and create a function to generate a batch of latent inputs.
</div>

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

#https://discuss.pytorch.org/t/how-do-i-print-output-of-each-layer-in-sequential/5773/3
class PrintLayer(nn.Module):
    def __init__(self,f):
        self.f = f
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        if self.f:
            print(x.shape)
        return x

class Discriminator(nn.Module):
    def __init__(self, ngpu, f=False):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            PrintLayer(f),
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            PrintLayer(f),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            PrintLayer(f),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            PrintLayer(f),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            PrintLayer(f),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
            PrintLayer(f)
        )

    def forward(self, input):
        return self.main(input)

class Generator(nn.Module):
    def __init__(self, ngpu, f=False):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            PrintLayer(f),
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            PrintLayer(f),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            PrintLayer(f),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            PrintLayer(f),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            PrintLayer(f),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( 64,nc, 4, 2, 1, bias=False),
            nn.Tanh(),
            PrintLayer(f)
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Create the generator
netG = Generator(ngpu, True).to(device)

In [ ]:
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

In [ ]:
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu, True).to(device)

In [ ]:
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

In [ ]:
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

# Preparing to train

<div style="background-color: #e6ccff; padding: 10px;">
    
Now we've created our generator and discriminator, we need to train them! As it stands, they've been initiated with random weights for all trainable parameters. This means that initially, both the generator and discriminator are going to be terrible at their respective tasks.

By training them, both will improve, and through the adversarial nature of the loss function, one network getting stronger *should* result in the other improving too. This isn't always the case however, sometimes either the generator or discriminator will become too strong and overpower the other. This is a failure case, and the model will no longer produce anything meaningful.

In the case of the generator winning, it learns a pattern that fools the discriminator while not necessarily representing the dataset at all. Often, the mode will collpase, with all latent variables converging to the same output image. If the discriminator wins, the generator loses its gradient and will begin to produce garbage outputs.

We want to avoid both cases, which makes GANs more difficult to train than most other machine learning models.

To begin our training, we first need to create an optimiser for each network. This optimiser implements an algorithm for performing backpropagation on the neural network. For this demo, we'll use the well known and effective Adam optimiser. If you want to know more about how this works, the paper it was proposed in can be found [here](https://arxiv.org/abs/1412.6980).
    
</div>

# Learning Rates

<div style="background-color: #e6ccff; padding: 10px;">

We set the initial learning rate for both optimisers to 0.0002. The learning rate is the single most important hyperparameter when training, and changing it can have drastic results. Try experimenting with much larger learning rates on both the generator and discriminator and see how it affects training!

</div>

<div style="background-color: #cce5ff; padding: 10px;">

    
**There is a link back to here at the end of this section of the tutorial.**
    
After you have gone through the tutorial rerun the section below with different learning rates to see the effects. 
    
The learning rates are initally set at:
```python
    learning_rate_dis = 0.0002
    learning_rate_gen = 0.0002
```    
    
For example try increasing or decreasing by a factor of 10. What happens if you alter one rate but not the other?   
    
</div>

In [11]:
# Number of training epochs
num_epochs = 1000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5



In [ ]:
# Set random seed for reproducibility
# manualSeed = 999
manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

<div style="background-color: #e6ccff; padding: 10px;">
    
We need to define the loss function that we're going to use to train our networks. We'll use binary cross entropy to measure our discriminator's predictions.

If we input real images into the discriminator we expect it to output 1 (100% certainty that the image is real).

The further away it is from 1 and the closer it is to 0 the more we should penalize it, as it is making an incorrect prediction.

BCE loss essentially becomes -log(x) when the target label is 1.

Similarly for fake images, the target label is 0 (as we want the discriminator to output 0 for fake images) and we want to penalise the generator if it starts outputing values close to 1. So we basically want to mirror the above loss function and that's just: -log(1-x).

BCE loss basically becomes -log(1-x) when it's target label is 0.
    
</div>    

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

In [ ]:
# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
print(fixed_noise.shape)
print(fixed_noise.max())
print(fixed_noise.min())

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


<div style="background-color: #ccffcc; padding: 10px;">
Finally, let's define a few variables to help our training process.
</div>

In [ ]:


# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

<div style="background-color: #ccffcc; padding: 10px;">
We're now ready to begin our training loop!
</div>

# Training

<div style="background-color: #ccffcc; padding: 10px;">
Let's create our GAN training loop. We start with training the discriminator, as this helps to prevent early mode collapse. 
</div>   

<div style="background-color: #ffcdcc; padding: 10px;">     

 **For 100 epochs, this training will take approximately 20-25 minutes, go grab a coffee!**

</div>  

In [ ]:
# Training Loop
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device).unsqueeze(1)
        b_size = real_cpu.size(0)
        # print(b_size)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = torch.squeeze(netD(real_cpu))
        # Calculate loss on all-real batch
        # print(output)
        # print(output.shape)
        # print(label)
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        # print(fake.shape)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                % (epoch, num_epochs, i, len(dataloader),
                    errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu().numpy()
            img_list.append(fake)
            np.save("produced_small.npy",img_list)

        iters += 1

<div style="background-color: #ccffcc; padding: 10px;">

Now our training is finished, it's time to evaluate how it's doing!

If you'd like, you can check out the image samples we saved during training. Remember we created a set of constant latent inputs, meaning each image has the same input and differs only because the generator's weights have updated.

The images folder contains samples produced as training has progressed. Can you see how it's improving over time?

</div>

# Evaluating the GAN

<div style="background-color: #ccffcc; padding: 10px;">

Now we've trained our model, we can use it to generate a new handwritten digit based on the MNIST dataset, but totally unique!

To start with, we'll define a few functions for prettying up and displaying the sample images.
    
</div>    

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig("loss.png")

<div style="background-color: #ccffcc; padding: 10px;">

Now we've spent all that time training the model we can save ourselves time in the future by saving the trained model
    
</div>

In [ ]:
PATH = "netG.pt"
torch.save(netG, PATH)

PATH = "netD.pt"
torch.save(netD, PATH)

In [ ]:
generator_net.eval()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
HTML(ani.to_jshtml())
        